<a href="https://colab.research.google.com/github/AINERD007/AINERD007/blob/main/Boltzmann_Machine_for_a_binary_image_reconstruction_task_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Boltzmann Machine and train it using the Contrastive Divergence algorithm for a binary image reconstruction task.**

In [1]:
pip install numpy


In [2]:
import numpy as np


In [3]:
class BoltzmannMachine:
    def __init__(self, num_visible, num_hidden):
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.weights = np.random.normal(loc=0.0, scale=0.01, size=(num_visible, num_hidden))
        self.visible_bias = np.zeros(num_visible)
        self.hidden_bias = np.zeros(num_hidden)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def energy(self, visible_state, hidden_state):
        return -np.dot(visible_state, self.visible_bias) - np.dot(hidden_state, self.hidden_bias) - np.dot(
            np.dot(visible_state, self.weights), hidden_state)

    def sample_hidden(self, visible_state):
        hidden_probabilities = self.sigmoid(np.dot(visible_state, self.weights) + self.hidden_bias)
        return np.random.binomial(1, hidden_probabilities)

    def sample_visible(self, hidden_state):
        visible_probabilities = self.sigmoid(np.dot(hidden_state, self.weights.T) + self.visible_bias)
        return np.random.binomial(1, visible_probabilities)

    def gibbs_sampling(self, visible_state, k=1):
        for _ in range(k):
            hidden_state = self.sample_hidden(visible_state)
            visible_state = self.sample_visible(hidden_state)
        return visible_state

    def train(self, data, epochs=10, learning_rate=0.1, k=1):
        num_samples = data.shape[0]
        for epoch in range(epochs):
            for sample in range(num_samples):
                visible_state = data[sample]
                hidden_state = self.sample_hidden(visible_state)
                reconstructed_visible = self.gibbs_sampling(visible_state, k)
                self.weights += learning_rate * (
                        np.outer(visible_state, hidden_state) - np.outer(reconstructed_visible, hidden_state))
                self.visible_bias += learning_rate * (visible_state - reconstructed_visible)
                self.hidden_bias += learning_rate * (hidden_state - self.sample_hidden(reconstructed_visible))

    def reconstruct(self, visible_state, steps=100):
        for _ in range(steps):
            hidden_state = self.sample_hidden(visible_state)
            visible_state = self.sample_visible(hidden_state)
        return visible_state


In [4]:
# Sample Data
data = np.array([[1, 0, 0, 1, 1],
                 [1, 0, 1, 1, 0],
                 [1, 1, 0, 0, 1],
                 [0, 1, 1, 0, 0]])

num_visible = data.shape[1]
num_hidden = 3


In [5]:
# Create and Train Boltzmann Machine
bm = BoltzmannMachine(num_visible, num_hidden)
bm.train(data, epochs=100, learning_rate=0.1, k=1)


In [6]:
# Reconstruct Images
for i in range(len(data)):
    original_image = data[i]
    reconstructed_image = bm.reconstruct(original_image)
    print(f"Original Image: {original_image}")
    print(f"Reconstructed Image: {reconstructed_image}")
    print()


Original Image: [1 0 0 1 1]
Reconstructed Image: [1 0 1 1 1]

Original Image: [1 0 1 1 0]
Reconstructed Image: [0 1 1 0 0]

Original Image: [1 1 0 0 1]
Reconstructed Image: [0 1 1 0 1]

Original Image: [0 1 1 0 0]
Reconstructed Image: [1 0 0 1 0]

